In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Data Pre-Processing

Before we can use our aviation data to train an accident severity classifier, we must first extract the most relevant information from it and convert it into a format that our future model can understand.

In order to accomplish this, we will perform a series of pre-processing steps designed to clean and re-format our aviation dataset.

## Data Cleaning

To begin with, we will import our stored model and convert all of its columns to lowercase for ease of use in Python.

In [2]:
# Importing dataset
accident_df = pd.read_pickle('./data/cm_vehicles_flattened_joined')
accident_df.columns = accident_df.columns.str.lower()

### Removing NaNs from Columns of Interest

The first thing that we must do is identify the columns of data that are of interest to us and remove or replace any missing values (also known as `NaN`s) from them. Having any `NaN` values in our dataset will make it impossible for our future model to understand and accept our data for training. As such, it is crucial that they be addressed beforeheand.

However, we should not simply remove all `NaN`s from our dataset. If we remove any row of data that has a `NaN` in it, we will be losing a significant amount of data that would otherwise be in the other columns of those rows. To prevent an extreme loss of data, we should only focus on cleaning the columns that we plan to use in our training.

Since our model is attempting to determine the factors that affect accident severity, we can use the probable cause reports and analysis narrative reports reports for each accident to train our model on the aspects that distinguish each accident. We should _not_ include the factual narrative report however, as it contains too much text and would make our dataset too large to store in a notebook such as this for use in training.

In addition to this text data, we can train our model on the latitude, longitude, time, state, and plane manufacturer (make) of each accident. These contextual factors for each accident are commonly believed to play a significant role in accident severity, so they should be included in our model so we can infer whether or not these factors are as important as many believe them to be.

In [3]:
# Defining columns of interest
text_columns_of_interest = ['cm_probablecause','analysisnarrative']
non_text_columns_of_interest = ['cm_latitude', 'cm_longitude', 'cm_eventdate', 'cm_state', 'make']

Having selected our columns of interest, with an eye towards minimizing the number of columns we use to avoid overfitting our model later on, we can inspect the `NaN`s present in each column to determine whether or not to remove or replace them.

In the cell below, we can see that the `cm_probablecause` column has an extremely small number of `NaN`s (<0.1% of the dataset). As such, we can safely remove these `NaN`s without meaningfully affecting the distribution of our data.

In [4]:
# Displaying presence of NaN values in key text columns
print(f'The probable cause column has {accident_df.cm_probablecause.isna().sum()} NaN values.')
print(f'The analysis narrative column has {accident_df.analysisnarrative.isna().sum()} NaN values.')
print(f'The factual narrative column has {accident_df.factualnarrative.isna().sum()} NaN values.')

The probable cause column has 14 NaN values.
The analysis narrative column has 0 NaN values.
The factual narrative column has 0 NaN values.


In [5]:
# Removing NaNs from columns with text
accident_df = accident_df.dropna(subset=text_columns_of_interest)

Moving on to the contextual feature columns, we can see that the `cm_latitide`, `cm_longitude`, and `cm_state` columns are each missing approximately 1% of the dataset's values. As this is an extremely small proportion of the data, these entries can be safely dropped without fear of misrepresenting our dataset.

In [6]:
# Displaying presence of NaN values in key non-desciptive columns
accident_df[non_text_columns_of_interest].isna().sum()

cm_latitude     413
cm_longitude    414
cm_eventdate      0
cm_state        104
make              9
dtype: int64

In [7]:
# Removing NaNs from columns without descriptive text
accident_df = accident_df.dropna(subset=non_text_columns_of_interest)

## Converting Data

Now that our data has been cleaned of `NaN` values, we can begin converting it into an AI-friendly format that will allow us to better train our model.

First, we can convert the date and time string in the `cm_eventdate` column from a string of text into the number of seconds that have elapsed since epoch (Jan, 1, 1970). This allows us to represent our datetime data as a simple integer value, which our future model will be better able to interpret.

In [8]:
# Converting accident date information from strings to seconds since epoch
accident_df.loc[:,'cm_eventdate'] = pd.to_datetime(accident_df.cm_eventdate,
                                                  format='%Y-%m-%dT%H:%M:%SZ').astype('int64')//1e9

/tmp/ipykernel_12150/301658041.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  accident_df.loc[:,'cm_eventdate'] = pd.to_datetime(accident_df.cm_eventdate,


### Separating Data

**Briefly explain the need for train/test splits**

In [9]:
# Splitting data into train and test sets as well as feature and target sets
accident_df_train, accident_df_test = train_test_split(accident_df)

In order to further prepare our data for use in our aviation model, we must separate it based on the kind of processing that it will need.

The columns `cm_probablecause` and `cm_analysisnarrative` both contain large quantities of string data that correspond to a report on the events of a given accident. In order to use this in a model, natural language processing (NLP) methods will need to be applied to convert these strings of text into vectors of word frequencies. As these NLP steps are fundamentally different from the preprocessing that will be required for the remainder of the data, we ought to separate these columns out.

In [10]:
# Selecting relevant data from dataset

# Separating out target data
accident_severity = accident_df_train[['cm_highestinjury']]
accident_severity_test = accident_df_test[['cm_highestinjury']]
# Separating out report data for NLP and vectorization
accident_df_text_columns = accident_df_train[text_columns_of_interest]
accident_df_text_columns_test = accident_df_test[text_columns_of_interest]
# Selecting relevant non-report columns
accident_df_trimmed = accident_df_train[non_text_columns_of_interest]
accident_df_trimmed_test = accident_df_test[non_text_columns_of_interest]
# Consider adding: cm_city

### Converting and Combining Data

When performing NLP preprocessing, it is important to collect all the text data that we are interested in into a single place. The process of NLP and vectorization undertaken in this project does not rely on the structure of any given sentence, and instead focuses on the frequency with which a word appears. As such, we can feel free to combine our probable cause and analysis narrative report columns into a single column without affecting the final result of our NLP.

In [11]:
# Combining text columns
accident_df_text = pd.DataFrame(accident_df_text_columns.cm_probablecause + " " + \
                   accident_df_text_columns.analysisnarrative)
accident_df_text_test = pd.DataFrame(accident_df_text_columns_test.cm_probablecause + " " + \
                                     accident_df_text_columns_test.analysisnarrative)

## Natural Language Pre-Processing

In this natural language processing phase, we will correct three issues that negatively affect the quality of our report data with regards to machine learning. These issues are stop words, conjugation, and the string format.

The vast majority of english sentences contain numerous words that mean little in and of themselves and serve only to maintain the grammar and syntax of an english sentence. Such words include 'is' and 'to'. These words are known as stop words, and our fture model will attempt to train on them if they are not removed from our dataset. Since these words to not actually carry any meaning, our model's attempts to fit to these words will only result in overfitting and increased training time, resulting in a worse model overall.

In addition to this, many english that _do_ have meaning come in multiple forms depending on the syntax of the sentence around it. Verb conjugation and plural forms are both examples of this. If multiple versions of the same word are allowed to exist in our dataset, our model will attempt to train on each of them as if they were completely different words. This would lead to model attempting to fit to multiple highly correlated features and dividing up its weights across the various forms of a word that appear in the dataset. In order to combat this, we can attempt to convert every word in our dataset to its syntactically neutral form, known as a lemma, through the process of lemmatization.

And lastly, all of our report data is represented in string format. That is to say, as blocks of text. This is a significant issue to us, as our machine learning model is incapable of training on any data that is not purely numeric. In order to use our accident report text data in our machine learning model, we need to convert it into a numeric format. This can be accomplished through the process of vectorization, which creates a column for each unique word in our dataset and stores the frequency with which that word appears in each row of our dataset. In this format, our future model will be perfectly capable of training on our report data and inferring the factors that contribute most to accident severity.

The following three cells remove stop words from our dataset, lemmatize the remaining words, and vectorize the remaining lemmas, in that order.

In [12]:
def get_relevant_lemmas(text):
    '''
    This function takes in a string of text and returns that same string
    with all english stop words removed and all remaining words lemmatized.
    
    Parameters
    ----------
    text: A string of english text.
    
    Returns
    -------
    A lemmatized string of english text with no stop words.
    '''
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    return " ".join([lemmatizer.lemmatize(word) for word in text.split() if not word.lower() in stop_words])

In [16]:
# Lemmatizing text data and removing stop words
accident_df_text_lemmas = accident_df_text.applymap(get_relevant_lemmas)
accident_df_text_test_lemmas = accident_df_text_test.applymap(get_relevant_lemmas)

In [17]:
# Vectorizing lemmatized text data and storing it as a DataFrame
tfidf = TfidfVectorizer()
tfidf.fit(accident_df_text_lemmas[0])
accident_df_tfidf = pd.DataFrame(tfidf.transform(accident_df_text_lemmas[0]).todense(),
                                 columns = tfidf.get_feature_names_out())
accident_df_tfidf_test = pd.DataFrame(tfidf.transform(accident_df_text_test_lemmas[0]).todense(),
                                      columns = tfidf.get_feature_names_out())

And with this, we have finished processing our accident report data. 

## Numeric and Categorical Pre-Processing

And with our report data addressed, we can move on to proccessing our contextual data. All of our contextual data falls into one of two groups: numeric data and categorical data. Both of which have their own issues that must be corrected to maximize future model performance.

Numeric data such as latitude, longitude, and event time are all represented purely as numbers. Since data types are already compatible with our machine learning models, they theoretically do not need to be modified in order to be used. However, our future models will make use of a group of techniques known as 'regularization' to reduce overfitting and improve performance, and regularization is sensitive to the scale of the data given to the model. To maximize the accuracy of our final model, it would there fore be benefitial to compress all of our numeric data down to one order of magnitude while preserving the variance within the data that our model needs in order to learn. This can be accomplished through the process of scaling.

By contrast, categorical data such as the state the accident took place in and the plane's manufacturer are all represented as strings of text, like with our reports. However, these simple strings of text do not need to be pre-processed with NLP. Instead, we can focus directly on encoding them in a numeric format. One extremely direct way is to simply create a column for each possible string of text within `cm_state` and `make` that contains a 1 if a row contained that string and 0 if it did not. This is known as One-Hot Encoding, and it is the simplest approach to numerically encoding nominal categorical data.

The following cell used a column transformer to scale numeric columns and encode categorical columns.

In [18]:
# One-Hot encoding non-NLP categorical data
# and scaling non-NLP numeric data
pre_processor = ColumnTransformer(
    [('scaler', StandardScaler(), ['cm_latitude', 'cm_longitude', 'cm_eventdate']),
     ('onehot', OneHotEncoder(drop = 'first', handle_unknown = 'ignore'), ['cm_state', 'make'])],
    verbose_feature_names_out = False
)
pre_processor.fit(accident_df_trimmed)
# Storing processed non-NLP data for merging with NLP data
accident_df_trimmed_transformed = pd.DataFrame(pre_processor.transform(accident_df_trimmed).todense(), 
                                               columns = pre_processor.get_feature_names_out())
accident_df_trimmed_transformed_test = pd.DataFrame(pre_processor.transform(accident_df_trimmed_test).todense(), 
                                                    columns = pre_processor.get_feature_names_out())

/home/brunobarreto/anaconda3/envs/work/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


### Combining and Saving Data

With our separated datasets now fully processed, we can re-combine them to obtain the fully prepared data that we will use in the training our aviation accident severity model.

In [21]:
# Combining datasets
accident_df_train = pd.concat([accident_df_trimmed_transformed, 
                               accident_df_tfidf], axis = 1)
accident_df_test = pd.concat([accident_df_trimmed_transformed_test, 
                              accident_df_tfidf_test], axis = 1)

And we conclude by saving our datasets, in pickle format, for use in our modelling notebook.

In [22]:
# Saving data for use in modelling
accident_df_train.to_pickle('./data/train')
accident_df_test.to_pickle('./data/test')
accident_severity.to_pickle('./data/train_severity')
accident_severity_test.to_pickle('./data/test_severity')